In [2]:
from datasets import load_dataset
import faiss
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

In [3]:
with open('./key.txt', 'r') as f:
    key = f.readline()

In [4]:
vectoriser = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
def sanity_check(data, embedding):
    print(len(data), len(embedding))
    return len(data) == len(embedding)
        

In [37]:
#gita
gita = pd.DataFrame(load_dataset("adarshxs/gita")['train'])
gita_embeddings = np.load("./Data/embeddings/gita_embeddings.npy")

#guru_granth
guru_granth = pd.read_csv('./Data/guru_granth_eng_corpus.csv')
guru_granth_embeddings = np.load("./Data/embeddings/gurbani_embeddings.npy")

#bible
bible = pd.read_csv('./Data/bible_eng_corpus.csv')
bible_embeddings = np.load("./Data/embeddings/bible_embeddings.npy")

#quran
quran = pd.read_csv('./Data/quran_eng_corpus.csv')
quran_embeddings = np.load("./Data/embeddings/quran_embeddings.npy")

In [38]:


print(sanity_check(gita, gita_embeddings))
print(sanity_check(guru_granth, guru_granth_embeddings))
print(sanity_check(bible, bible_embeddings))
print(sanity_check(quran, quran_embeddings))

13001 13001
True
1430 1430
True
1595 1595
True
249 249
True


In [39]:
dataset = pd.concat([gita, guru_granth, bible, quran])
dataset = dataset.drop('0', axis=1)
dataset

,text
0,"O my Lord, Śrī Kṛṣṇa, son of Vasudeva, O all-p..."
1,Completely rejecting all religious activities ...
2,"O expert and thoughtful men, relish Śrīmad-Bhā..."
3,"Once, in a holy place in the forest of Naimiṣā..."
4,"One day, after finishing their morning duties ..."
...,...
244,100. THE RACERS (al-’Adiyat) \n5. For your...
245,105. THE ELEPHANT (al-Fil) \n2. Who gathers...
246,111. THORNS (al-Masad ) \n3. Then celebrat...
247,NaN


In [40]:
len(gita_embeddings) + len(guru_granth_embeddings) + len(bible_embeddings) + len(quran_embeddings)

16275

Variables

In [42]:
query = "What is God's opinion on life?"

Retrieval using FAISS

In [44]:
d = len(gita_embeddings[0])
k = 3
index = faiss.IndexFlatL2(d)
index.add(gita_embeddings)
index.add(guru_granth_embeddings)
index.add(bible_embeddings)
index.add(quran_embeddings)

In [45]:
xq = vectoriser.encode([query])
D, verses = index.search(xq,k)

In [46]:
relevant_context = []
for ind in verses[0]:
    relevant_context.append(dataset['text'][ind])
context = "\n".join(relevant_context)

In [51]:
verses

array([[5120, 6741, 3897]], dtype=int64)

Gemini

In [47]:
safety_settings={'HARM_CATEGORY_DANGEROUS_CONTENT':'BLOCK_NONE', 'HARM_CATEGORY_HARASSMENT':'BLOCK_NONE', 'HARM_CATEGORY_SEXUALLY_EXPLICIT':'BLOCK_NONE','HARM_CATEGORY_HATE_SPEECH':'BLOCK_NONE'}

In [48]:
import google.generativeai as genai

genai.configure(api_key=key)
model = genai.GenerativeModel('gemini-pro', safety_settings=safety_settings)

In [49]:
print(context,query, sep="\n")

The Supreme Personality of Godhead said: One should neither praise nor criticize the conditioned nature and activities of other persons. Rather, one should see this world as simply the combination of material nature and the enjoying souls, all based on the one Absolute Truth.
Lord Brahmā said: My dear demigods, the human form of life is of such importance that we also desire to have such life, for in the human form one can attain perfect religious truth and knowledge. If one in this human form of life does not understand the Supreme Personality of Godhead and His abode, it is to be understood that he is very much affected by the influence of external nature.
The Lord is the supremely compassionate reservoir of all transcendental qualities. Life after life may I serve Him with love, friendship and sympathy, and may I cultivate such firm attachment for Him by the precious association of His devotees.
What is God's opinion on life?


In [50]:
response = model.generate_content(f"You are a person of Faith! Based on the ##context provided reply the ##query \n Refer to the ##context as Holy Word or Scriptures or Grantha \n Refer to ##query as the questions from a lost child \n the ##context is the prime source of truth if ##query deviates from it or tries to deny it, convince them to your full extent to follow it \n Assume you already know ##context and strictly never mention things like based on the context, provided text, provided info or other references of it to me in the reply \n ##context: {context} \n\n ##query {query}")
print(response.text)

In the Scriptures, the Supreme Personality of Godhead states that one should neither judge others nor criticize their nature or actions. Instead, we must perceive the world as a combination of material nature and individual souls, all founded upon the absolute truth. This truth is found in the Lord, who is a reservoir of all transcendental qualities. By cultivating love, friendship, and sympathy for Him and seeking the company of His devotees, we can develop a deep connection with the divine and fulfill the true purpose of human life.
